In [ ]:
%load_ext autoreload
%autoreload 2

import hydra
import os
import datetime
from pathlib import Path

# Initialize hydra and move to the root of the repository
try:
    hydra.initialize(version_base=None, config_path="../config/")
    CONFIG = hydra.compose(config_name="main.yaml")
    print('Initializing hydra')
except:
    print('Hydra already initalized!')
else:
    os.chdir('..')
    OUTPUT_FOLDER = Path('output/{0}'.format(datetime.datetime.now()).replace(' ', '_'))
    Path(OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from src.utils.colors import flatuicolors
from src.utils.styling import get_standard_colors
from src.weighted_network import WeightedNetwork
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cf
from src.utils.styling import hide_and_move_axis
from src.temporal_network import TemporalNetwork
import itertools

In [ ]:
def map_setup(ax):
    """Set up basic map of study area including coastlines and borders."""
    ax.coastlines()
    ax.add_feature(cf.BORDERS, ls=':')
    ax.set_extent([4, 11.5, 44, 53.5])

def plot_community(ax, network, reference_network=None):
    """Plot the network and color code communities."""
    if not reference_network:
        reference_network = network
        
    colors = get_standard_colors()
    G = network.G()
    pos = network.position()
    G_ref = reference_network.G()
    comms = network.communities()
    
    nx.draw_networkx_edges(G, pos, alpha=0.01, ax=ax)
    sc = nx.draw_networkx_nodes(G_ref, pos, node_size=8, node_color='w', alpha=0.25, ax=ax)
    sc.set_edgecolor('k')
    
    for i, comm in enumerate(comms):
        sc = nx.draw_networkx_nodes(G, pos, nodelist=comm, node_size=12, node_color=colors[i], ax=ax)
        sc.set_edgecolor('k')
        sc.set_linewidth(.15)

def load(year0=None, year1=None, edge_filter='S238'):
    """Load the weighted network."""
    input_path = Path(CONFIG.data.raw)
    nodes_file = input_path / CONFIG.data.filenames.nodes
    edges_file = input_path / CONFIG.data.filenames.edges
    return WeightedNetwork(nodes_file=nodes_file, edges_file=edges_file, year0=year0, year1=year1, edge_filter=edge_filter)


def load_temporal(edge_filter='S238'):
    """Load the temporal network."""
    input_path = Path(CONFIG.data.raw)
    nodes_file = input_path / CONFIG.data.filenames.nodes
    edges_file = input_path / CONFIG.data.filenames.edges
    return TemporalNetwork(nodes_file=nodes_file, edges_file=edges_file, edge_filter=edge_filter)

def finalize(filename):
    """Crop and convert output figures."""
    os.system(f'pdfcrop --margin 5 {filename} {filename}')
    os.system(f'convert -density 400 {filename} {filename}.jpg')


def plot_temporal_evolution_of_governance(network, normalize=True):

    fig, axarr = plt.subplots(2, 5, sharey=True, sharex=True, figsize=(8, 4))
    flatax = axarr.flatten()
    colors = get_standard_colors()
    
    communities = network.communities()
    edges = network.edges()
        
    for i, community in enumerate(communities):
    
        in_edges = edges[edges['from'].isin(community) & edges['to'].isin(community)]
        timeseries = in_edges.groupby('Year').ruling_party_category.value_counts().unstack().fillna(0)
    
        if 'R' not in timeseries.columns:
            timeseries['R'] = 0
        if 'S' not in timeseries.columns:
            timeseries['S'] = 0

        if normalize:
            norm = timeseries.S + timeseries.R
            label = 'Share of links\nin Community'
            filename = 'timeseries_governance_relative.jpg'
        else:
            norm = 1
            label = 'Number of links\nin Community'
            filename = 'timeseries_governance_absolute.jpg'
            
        flatax[i].plot(timeseries.index, timeseries.S / norm , label='Secular', c=colors[i])
        flatax[i].plot(timeseries.index, timeseries.R / norm, label='Religious', c=colors[i], ls='--')
        hide_and_move_axis(flatax[i])
            
    flatax[i].legend()

    for ax in axarr[1, :]:
        ax.set_xlabel('Year')
    
    for ax in axarr[:, 0]:
        ax.set_ylabel(label)
    
    plt.tight_layout()

    plt.savefig(OUTPUT_FOLDER / filename, dpi=400)


def plot_pooled_sample(network):
    """Plot the network and its corresponding communities on a single map."""
    ax = plt.axes(projection=CRS)
    map_setup(ax)
    plot_community(ax, network)
    
    filename = OUTPUT_FOLDER / 'communities.pdf'
    plt.savefig(filename)
    finalize(filename)


def governing_parties_histograms(network):
    """Frequency histograms of religious and secular rulership in each community."""
    fig, axarr = plt.subplots(2, 5, sharey=True, sharex=True, figsize=(8, 4))
    flatax = axarr.flatten()
    
    edges = network.edges()
    comms = network.communities()
    colors = get_standard_colors()
    
    for i, community in enumerate(comms):
        
        in_edges = edges[edges['from'].isin(community) & edges['to'].isin(community)]
        ruling_parties = in_edges.PartyID.str[0]
        hist = ruling_parties.value_counts()
    
        hist = pd.DataFrame(hist)
        hist.sort_index(inplace=True)
        
        x = hist.index
        y = hist['count'].values
    
        y = y / y.sum()
        
        flatax[i].bar(x, y, color=colors[i])
        hide_and_move_axis(flatax[i])
    
    axarr[1, 2].set_xlabel('Type of governing party')
    axarr[0, 0].set_ylabel('Relative frequency')
    axarr[1, 0].set_ylabel('Relative frequency')
    plt.tight_layout()
    
    plt.savefig(OUTPUT_FOLDER / 'governing_parties.jpg', dpi=400)


def plot_temporal_network_statistics(network, hamming_threshold=400, min_year=1200):
    """Plot degree and hamming distance over time for the temporal network."""
    f, axarr = plt.subplots(2, 1, figsize=(5, 3.5), sharex=True)
    
    yrs = network.years()
    H = network.hamming_distance()
    K = network.average_degree_sequence()
    
    mask = yrs >= min_year
    yrs = yrs[mask]
    H = H[mask]
    K = K[mask]

    # Remove years that are multiples of 50, as there is artificially 
    # high hamming distance due to the data resolution
    mask1 = ((yrs - 1) % 50) != 0
    mask2 = H > hamming_threshold
    print('Years with significant changes:', yrs[mask1 & mask2])
    
    axarr[0].plot(yrs, K, c=flatuicolors.wetasphalt)
    axarr[1].plot(yrs[mask1], H[mask1], c=flatuicolors.wetasphalt)
    
    for ax in axarr:
        hide_and_move_axis(ax)
        for yr in yrs[mask1 & mask2]:
            ax.axvline(yr, zorder=0, c=flatuicolors.pomegranate, lw=4, alpha=0.3)
    
    axarr[1].axhline(400, ls=':', c='k')
    axarr[1].set_xlabel('Year')
    axarr[0].set_ylabel('Average Degree')
    axarr[1].set_ylabel('No. links that change')
    
    plt.tight_layout()
    plt.savefig(OUTPUT_FOLDER / 'temporal_evolution.png', dpi=400)


def get_temporal_communities(bins):

    # Get list of communities for each time slice
    networks = {}
    for year0, year1 in bins:
        G = load(year0=year0, year1=year1)
        networks[str(year0) + '-' + str(year1)] = G

    reference_key = str(bins[0][0]) + '-' + str(bins[0][1])
    reference_network = networks[reference_key]
    
    for year0, year1 in bins[1:]:

        key = str(year0) + '-' + str(year1)
        reference_communities = reference_network.communities()
        communities_to_sort = networks[key].communities()

        sorted_communities = []
        for oc in reference_communities:
            overlap = [len(oc.intersection(nc)) for nc in communities_to_sort]
            index = np.argmax(overlap)
            sorted_communities.append(communities_to_sort[index])
            communities_to_sort = communities_to_sort[:index] + communities_to_sort[index+1:]
    
        for nc in communities_to_sort:
            sorted_communities.append(nc)

        networks[key].set_communities(sorted_communities)
        reference_network = networks[key]
        
    return networks


def plot_temporal_communities(bins, networks, reference_network, nrows=2, ncols=3, filename='temporal_communities.pdf', axis_order=None):

    f, axarr = plt.subplots(nrows, ncols, subplot_kw={'projection': CRS}, figsize=(2.5 * ncols * 1.25, 4 * nrows * 1.25))
    plt.subplots_adjust(wspace=.05, hspace=.1)
    
    axarr = axarr.flatten()
    if axis_order is not None:
        axarr = axarr.flatten()[axis_order]
    
    for bin, ax in zip(bins, axarr):
        key = f'{bin[0]}-{bin[1]}'
        ax.set_title(key)
        map_setup(ax)
        plot_community(ax, networks[key], reference_network=reference_network)
    
    filename = OUTPUT_FOLDER / filename
    plt.savefig(filename)
    finalize(filename)


def plot_community_characteristics(network, title=None):
   
    fig, axarr = plt.subplots(2, 5, sharey=True, figsize=(10, 4))
    output_file = f'governing_parties_distinct_{title}.jpg'

    flatax = axarr.flatten()
    colors = get_standard_colors()

    communities = network.communities()
    edges = network.edges()

    for i, community in enumerate(communities):

        in_edges = edges[edges['from'].isin(community) & edges['to'].isin(community)]
        
        hist = in_edges.PartyID.value_counts()
        hist.sort_index(inplace=True)

        x, y = hist.index, hist.values
        y = y / y.sum()

        flatax[i].bar(x, y, color=colors[i])
        hide_and_move_axis(flatax[i])

        flatax[i].set_xticks(x, x, rotation='vertical', size=6)

    axarr[1, 2].set_xlabel('Governing party')
    axarr[0, 0].set_ylabel('Relative frequency')
    axarr[1, 0].set_ylabel('Relative frequency')

    if title:
        fig.suptitle(title, fontsize=16)

    plt.tight_layout()
    plt.savefig(OUTPUT_FOLDER / output_file, dpi=400)


def get_coocurrences(network, require_edge=False):

    nodes = network.nodes()
    communities = network.communities()
    
    if require_edge:
        edges = network.edges()    
        df = pd.merge(edges, nodes, left_on=['from', 'Year'], right_on=['PlaceID', 'Year'])
        df.drop(columns=['PartyID', 'ruling_party_category', 'Unnamed: 0_x', 'Unnamed: 0_y', 'PlaceID', 'XCOORD', 'YCOORD', 'PlaceName'], inplace=True)

    else:
        edges = list(itertools.combinations(nodes.PlaceID, 2))
        #for comm in communities:
        #    edges += list(itertools.combinations(comm, 2))
        edges = pd.DataFrame(edges, columns=['from', 'to'])
        df = pd.merge(edges, nodes, left_on=['from'], right_on=['PlaceID'])
        df.drop(columns=['Unnamed: 0', 'PlaceID', 'XCOORD', 'YCOORD', 'PlaceName'], inplace=True)

    df = pd.merge(df, nodes, left_on=['to', 'Year'], right_on=['PlaceID', 'Year'])
    df.drop(columns=['Unnamed: 0', 'PlaceID', 'XCOORD', 'YCOORD', 'PlaceName'], inplace=True)
        
    df['co-occur'] = (df.Juden_x & df.Juden_y).astype(bool)
    df['coherence'] = df.Juden_x == df.Juden_y
    df['fromto'] = df['from'].astype(str) + '_' + df['to'].astype(str)

    return df


def get_shared_status(network):
    nodes = network.nodes()
    places = nodes.PlaceID.unique()
    
    frames = []
    for p_id in places:
        print(p_id, end='\r')
        frames.append(pd.merge(nodes[nodes.PlaceID == p_id], nodes[nodes.PlaceID > p_id], on='Year')[['PlaceID_x', 'PlaceID_y', 'Year', 'Juden_x', 'Juden_y']])
    
    df = pd.concat(frames)
    df['co-occur'] = (df.Juden_x & df.Juden_y).astype(bool)
    df['coherence'] = df.Juden_x == df.Juden_y
    df['fromto'] = df['PlaceID_x'].astype(str) + '_' + df['PlaceID_y'].astype(str)

    return df


def plot_shared_status(shared_status_df, key = 'coherence', n_bins = 5, min_values = 10):
    
    f, axarr = plt.subplots(2, 5, sharex=True, sharey=True, figsize=(8, 4.5))
    color = get_standard_colors()

    bw = 1/n_bins

    if key == 'co-occur':
        label = 'Probability\nco-presence of Jews'
    elif key == 'coherence':
        label = 'Probability\nof coherence'

    outfile = OUTPUT_FOLDER / f'prob_{key}_commnuity_structure.jpg'

    # Plot probability of shared status within community
    for i, ax in enumerate(axarr.flatten()):
    
        community = G_WEIGHTED.communities()[i]
    
        e = shared_status_df[shared_status_df['PlaceID_x'].isin(community) & shared_status_df['PlaceID_y'].isin(community)]
        g = e.groupby('fromto')[key].agg(['mean', 'count'])
        g = g[g['count'] > min_values]

        count, bins = np.histogram(g['mean'], bins=np.linspace(0, 1, n_bins+1))
        count = count / count.sum()
        bins = .5 * (bins[1:] + bins[:-1])
        ax.bar(bins - .4 * bw + 0.015, count, width=bw * .4, color=color[i], alpha=0.95)

    # Plot probabiblity of shared status between any pair of nodes
    g = shared_status_df.groupby('fromto')[key].agg(['mean', 'count'])
    g = g[g['count'] > min_values]
    
    count, bins = np.histogram(g['mean'], bins=np.linspace(0, 1, n_bins+1))
    count = count / count.sum()
    bins = .5 * (bins[1:] + bins[:-1])
    
    for ax in axarr.flatten():
        ax.bar(bins - 0.015, count, width=bw * .4, color='k', alpha=0.25, zorder=0)

    # Finalize plot
    for ax in axarr[1]:
        ax.set_xlabel(label)

    for ax in axarr[:, 0]:
        ax.set_ylabel('Frequency')

    plt.tight_layout()
    plt.savefig(outfile)


def timeseries_percentage_point_changes(network):

    f, axarr = plt.subplots(2, 5, sharex=True, sharey=True, figsize=(8, 5))

    nodes = network.nodes()
    color = get_standard_colors()
    
    for i, ax in enumerate(axarr.flatten()):
        
        c = network.communities()[i]
        data = nodes[nodes.PlaceID.isin(c)].groupby('Year')['Juden'].agg(['mean', 'sum', 'count'])
        assert (data.index.diff().fillna(1) == 1).all()
        
        data['diff'] = data['mean'].diff() * 100
        
        data = data[data['count'] > (.25 * len(c))]
    
        mask = ((data.index - 1 ) % 50) != 0
        data = data[mask]
    
        data = data['diff']
        data.plot(ax=ax, c=color[i])
    
        for year, value in data[data < -20].items():
            ax.text(year+5, value, str(year - 1), ha='center', va='top')
            
        for year, value in data[data > 20].items():
            ax.text(year+5, value, str(year - 1))
        
        hide_and_move_axis(ax)
    
    for ax in axarr[:, 0]:
        ax.set_ylabel('Percentage point change\nin presence of Jews')
    plt.tight_layout()
    plt.savefig(OUTPUT_FOLDER / 'percentage_point_changes.jpg')


def plot_change_in_total_presence_of_jews(network):

    f, ax = plt.subplots()
    color = get_standard_colors()
    
    df = None
    for i, c in enumerate(network.communities()):
        data = nodes[nodes.PlaceID.isin(c)].groupby('Year')['Juden'].agg(['mean', 'sum', 'count'])
        data = data['sum'].diff()
        assert (data.index.diff().fillna(1) == 1).all()
        if df is None:
            df = pd.DataFrame(data).rename(columns={'sum': 'c0'})
        else:
            df = pd.merge(df, data, on='Year', how='outer').rename(columns={'sum': f'c{i}'})
    
    mask = ((df.index - 1 ) % 50) != 0
    df = df[mask]
    df = df.fillna(0)
    df = df[((df < -3) | (df > 3)).any(axis=1)]
    
    df_below = df.copy()
    df_below[df_below > 0] = 0
    df_above = df.copy()
    df_above[df_above < 0] = 0
    
    bottom_below = np.zeros(len(df))
    bottom_above = np.zeros(len(df))
    
    for i in range(10):
        data = df_below[f'c{i}'].copy()
        p = ax.bar(range(len(data)), data, bottom=bottom_below, color=color[i])
        bottom_below += data
    
        data = df_above[f'c{i}'].copy()
        p = ax.bar(range(len(data)), data, bottom=bottom_above, color=color[i])
        bottom_above += data
    
    hide_and_move_axis(ax)
    ax.set_xticks(range(len(data)), data.index - 1)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(0, c='k')
    ax.set_ylabel('Total change in cities with Jews')
    
    plt.tight_layout()
    plt.savefig(OUTPUT_FOLDER / 'total_change_jews.jpg')

In [ ]:
G_WEIGHTED = load()
G_TEMPORAL = load_temporal()

CRS = ccrs.LambertAzimuthalEqualArea(
    central_latitude=CONFIG.grid.central_latitidue, 
    central_longitude=CONFIG.grid.central_longitude, 
    false_easting=CONFIG.grid.false_easting, 
    false_northing=CONFIG.grid.false_northing
)

In [ ]:
# Plot the pooled sample network
plot_pooled_sample(G_WEIGHTED)

# Plot temporal evolution of shares of secular and religious government entities in pooled sample
plot_temporal_evolution_of_governance(G_WEIGHTED)
plot_temporal_evolution_of_governance(G_WEIGHTED, normalize=False)
governing_parties_histograms(G_WEIGHTED)

# Plot statistics of the temporal network 
plot_temporal_network_statistics(G_TEMPORAL)

In [ ]:
BINS = [(1351, 1354), (1355, 1400), (1401, 1414), (1415, 1417), (1415, 1417), (1418, 1450)]
networks = get_temporal_communities(BINS)
plot_temporal_communities(BINS, networks, G_WEIGHTED, filename='temporal_communities_hamming.pdf', axis_order=[0, 3, 1, 2, 4, 5])
plot_community_characteristics(networks['1401-1414'], title='1401-1414')
plot_community_characteristics(networks['1415-1417'], title='1415-1417')
plot_community_characteristics(networks['1418-1450'], title='1418-1450')

BINS = [(1251, 1300), (1301, 1400), (1400, 1520)]
networks = get_temporal_communities(BINS)
plot_temporal_communities(BINS, networks, G_WEIGHTED, nrows=1, ncols=3, filename='temporal_communities_slices.pdf')

In [ ]:
df = get_shared_status(G_WEIGHTED)
plot_shared_status(df, key='coherence')
plot_shared_status(df, key='co-occur')

In [ ]:
timeseries_percentage_point_changes(G_WEIGHTED)
plot_change_in_total_presence_of_jews(G_WEIGHTED)

# Sandbox

## Time series for share of cities with presence of Jews per community

In [ ]:
f, axarr = plt.subplots(2, 5, sharex=True, sharey=True, figsize=(8, 5))

for i, ax in enumerate(axarr.flatten()):
    c = G_WEIGHTED.communities()[i]
    data = nodes[nodes.PlaceID.isin(c)].groupby('Year')['Juden'].agg(['mean', 'sum', 'count'])
    data = data[data['count'] > (.25 * len(c))]
    data = data['sum'] / len(c)
    
    data.plot(ax=ax, c=color[i])
    
    hide_and_move_axis(ax)

for ax in axarr[:, 0]:
    ax.set_ylabel('Share of cities with\npresence of Jews')
plt.tight_layout()

## Video of temporal evolution

In [ ]:
import gif

@gif.frame
def plot_frame(year):
    ax = plt.axes(projection=CRS)
    map_setup(ax)
    ax.set_title(year)
    nx.draw_networkx_nodes(G_WEIGHTED.G(), G_WEIGHTED.position(), node_size=4, node_color='k', alpha=0.25)
    X, Y = nodes[(nodes.Year == year) & (nodes.Juden == 1)][['XCOORD', 'YCOORD']].drop_duplicates().values.T
    ax.scatter(X, Y)

nodes = G_WEIGHTED.nodes()
frames = []
for year in range(1000, 1521, 1):
    print(year, end='\r')
    frame = plot_frame(year)
    frames.append(frame)

gif.save(frames, "test.gif", duration=100)